 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Functions,-generators-and-data" data-toc-modified-id="Functions,-generators-and-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Functions, generators and data</a></span></li><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Training" data-toc-modified-id="Training-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training</a></span></li></ul></li><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Data-Augmentation" data-toc-modified-id="Data-Augmentation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Augmentation</a></span></li><li><span><a href="http://localhost:8889/notebooks/21-full-res-model-all-angles-vertical-cut-no-bbox.ipynb#Pseudo-labeling" data-toc-modified-id="Pseudo-labeling-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Pseudo-labeling</a></span></li></ul></div>

# Load libraries

In [ ]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

In [ ]:
import math
import random
import gzip
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Define loss functions

In [ ]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [ ]:
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [ ]:
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

In [ ]:
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [ ]:
def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    return model

In [ ]:
model = get_unet(input_shape=(768,1920,3), initial_filters=8)

In [ ]:
model.load_weights('./weights/best_weights_fullres3.hdf5')

# Training

## Functions, generators and data

In [ ]:
df_train = pd.read_csv('data/train_masks.csv')

In [ ]:
from os import listdir
ids_train = [x.split('.')[0] for x in listdir('data/train')]

In [ ]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [ ]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [ ]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [ ]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask


In [ ]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [ ]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = cv2.imread('data/train/{}.jpg'.format(id))
                mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
                
                img1  = np.copy(img[0:768, :, :])
                img1  = np.concatenate([np.zeros((768,2,3), np.uint8), img1], axis=1)
                mask1 = np.copy(mask[0:768, :])
                mask1 = np.expand_dims(mask1, axis=2)
                mask1 = np.concatenate([np.zeros((768,2,1), np.uint8), mask1], axis=1)
                
                img2  = np.copy(img[512:1280, :, :])
                img2  = np.concatenate([img2, np.zeros((768,2,3), np.uint8)], axis=1)
                mask2 = np.copy(mask[512:1280, :])
                mask2 = np.expand_dims(mask2, axis=2)
                mask2 = np.concatenate([mask2, np.zeros((768,2,1), np.uint8)], axis=1)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
            
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

## Training

In [ ]:
train_batch_size = 2
val_batch_size   = 8

In [ ]:
model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

In [ ]:
max_epochs = 50

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/vert_cut.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

# Data Augmentation

In [ ]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                
                img1  = np.copy(img[:, 0:1024, :])
                mask1 = np.copy(mask[:, 0:1024])
                mask1 = np.expand_dims(mask1, axis=2)
                img2  = np.copy(img[:, 894:, :])
                mask2 = np.copy(mask[:, 894:])
                mask2 = np.expand_dims(mask2, axis=2)
                
                img1 = randomHueSaturationValue(img1,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img2 = randomHueSaturationValue(img2,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [ ]:
model.compile(optimizer=RMSprop(lr=0.00001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
model.load_weights('./weights/full_res_ang_09_cut.hdf5')

In [ ]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/full_res_ang_09_cut2.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

In [ ]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                
                img1  = np.copy(img[:, 0:1024, :])
                mask1 = np.copy(mask[:, 0:1024])
                img2  = np.copy(img[:, 894:, :])
                mask2 = np.copy(mask[:, 894:])
                
                img1 = randomHueSaturationValue(img1,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img2 = randomHueSaturationValue(img2,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                
                img1, mask1 = randomShiftScaleRotate(img1, mask1,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.15, 0.15),
                                                   rotate_limit=(-0, 0))
                
                img2, mask2 = randomShiftScaleRotate(img2, mask2,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.15, 0.15),
                                                   rotate_limit=(-0, 0))
                
                mask1 = np.expand_dims(mask1, axis=2)
                mask2 = np.expand_dims(mask2, axis=2)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [ ]:
model.load_weights('./weights/full_res_ang_09_cut2.hdf5')

In [ ]:
model.compile(optimizer=RMSprop(lr=0.00001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/full_res_ang_09_cut3.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

# Pseudo-labeling

In [ ]:
from os import listdir
ids_test = [x.split('.')[0] for x in listdir('data/preds09/')]

In [ ]:
def test_generator(test_batch_size):
    while True:
        this_ids_test = random.sample(ids_test, len(ids_test))
        for start in range(0, len(ids_test), test_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + test_batch_size, len(ids_test))
            ids_test_batch = this_ids_test[start:end]
            for id in ids_test_batch:
                img  = cv2.imread('data/test/{}.jpg'.format(id))
                mask = np.load('./data/preds09/' + id + '.npy')*255
                
                img1  = np.copy(img[:, 0:1024, :])
                mask1 = np.copy(mask[:, 0:1024])
                img2  = np.copy(img[:, 894:, :])
                mask2 = np.copy(mask[:, 894:])
                
                img1 = randomHueSaturationValue(img1,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img2 = randomHueSaturationValue(img2,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                
                img1, mask1 = randomShiftScaleRotate(img1, mask1,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.15, 0.15),
                                                   rotate_limit=(-0, 0))
                
                img2, mask2 = randomShiftScaleRotate(img2, mask2,
                                                   shift_limit=(-0.0625, 0.0625),
                                                   scale_limit=(-0.15, 0.15),
                                                   rotate_limit=(-0, 0))
                
                mask1 = np.expand_dims(mask1, axis=2)
                mask2 = np.expand_dims(mask2, axis=2)
                
                                
                x_batch.append(img1)
                y_batch.append(mask1)
                x_batch.append(img2)
                y_batch.append(mask2)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [ ]:
def pseudo_lab_gen():
    train_gen = train_generator(4)
    test_gen  = test_generator(2)
    while True:
        train_stuff = next(train_gen)
        if train_stuff[0].shape[0] != 4:
            train_stuff = next(train_gen)
        
        test_stuff  = next(test_gen)
        if test_stuff[0].shape[0] != 2:
            test_stuff  = next(test_gen)
        yield np.concatenate((train_stuff[0], test_stuff[0]), axis=0), np.concatenate((train_stuff[1], test_stuff[1]), axis=0)

In [ ]:
model.compile(optimizer=RMSprop(lr=0.00001), loss=bce_dice_loss, metrics=[dice_coeff])

In [ ]:
model.load_weights('./weights/full_res_ang_09_cut3.hdf5')

In [ ]:
max_epochs=20

In [ ]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/full_res_ang_09_cut4.hdf5',
                             save_best_only=True,
                             save_weights_only=True)]

history = model.fit_generator(generator=pseudo_lab_gen(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(4)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))